# Análise dos dados obtidos

In [1]:
# Imports
#  /home/bradachi/Documentos/GitHub/NLP_teste/NLP_ENV/bin/python -m pip install 
import requests
import json
import pprint
import pandas as pd
import lxml
import time
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
import string
import matplotlib.pyplot as plt 
from textblob import TextBlob
from bertopic import BERTopic
from PIL import Image
import numpy as np
from LeIA import SentimentIntensityAnalyzer
# from selenium import webdriver
# from webdriver_manager.firefox import GeckoDriverManager 
# from selenium.webdriver.firefox.service import Service 
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import NoSuchElementException


[nltk_data] Downloading package punkt to /home/bradachi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/bradachi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bradachi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/bradachi/Documentos/GitHub/NLP_teste/NLP_ENV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
perguntas = pd.read_csv("perguntas.csv")
perguntas = perguntas.drop_duplicates().reset_index(drop=True)
perguntas

,pergunta,resposta,id,data
0,Minha cervejeira é VN28AF Termostato sollatek ...,"Boa tarde, seu termostato saiu de linha o subs...",MLB44743999,27/11/2025
1,Esse termostato tem os 4 pinos?,"Boa tarde, são 6 pinos porem os que tem função...",MLB44743999,26/11/2025
2,Teria para o modelo VF56DB 220V,"Boa tarde, seu modelo usa termostato sollatek,...",MLB44743999,26/06/2025
3,Ok ele e 220v,O termostato do anúncio é bivolt.,MLB44743999,15/05/2025
4,Bom dia ele serve para o modelo vb15 metal frio,"Bom dia, sim temos programações para expositor...",MLB44743999,16/05/2025
...,...,...,...,...
56460,Olá é luz quente,"Olá Tudo bem? Não, a luz é branco neutro 4000k...",MLB46936892,02/09/2025
56461,Precisso da nota fiscal de este produto,Olá boa tarde! Tudo bem? Claro! Pode nos envia...,MLB46936892,23/06/2025
56462,"Muito obrigado pela atenção, E27 que preciso. ...",Deu certo o Sr. conseguiu utilizar?,MLB46936892,29/05/2025
56463,Comprei chegou mas a rosca não é E27 é menor,Boa noite! Tudo bem? Mas esse modelo não temos...,MLB46936892,28/05/2025


In [38]:
avaliacoes = pd.read_csv("avaliacoes.csv")
avaliacoes = avaliacoes.drop_duplicates().reset_index(drop=True)
avaliacoes

,texto,nota,id,data
0,Excelente produto!!.,5,MLB46241561,09 out. 2025
1,Muito bom.,5,MLB48090512,15 out. 2025
2,Muito bom produto.,5,MLB48090512,16 out. 2025
3,Produto bom. Sem retorno.,5,MLB48090512,20 nov. 2025
4,Produto confiável e original. Eu mesmo arrumei...,5,MLB29176649,27 out. 2025
...,...,...,...,...
17737,Maravilhosa.,5,MLB25814527,20 nov. 2025
17738,É elogiado por sua integração eficaz com siste...,5,MLB25814527,20 nov. 2025
17739,Veio certinho está funcionando bem.,5,MLB25814527,31 mar. 2025
17740,É amplamente elogiado por sua facilidade de us...,5,MLB24689330,20 nov. 2025


In [39]:
smartObjects = pd.read_csv("contexto.csv")

wifiObjects = smartObjects[smartObjects["Context_Id"] == "WITH_WI_FI"]
bltObjects = smartObjects[smartObjects["Context_Id"] == "WITH_BLUETOOTH"]

smartObjects = pd.concat([wifiObjects, bltObjects])

smartObjects = smartObjects[smartObjects["value_name"] == "Sim"]

smartObjects = smartObjects["id"].drop_duplicates().reset_index(drop=True)
smartObjects = list(smartObjects)
smartObjects

['MLB36333923',
 'MLB36139109',
 'MLB21345002',
 'MLB38703841',
 'MLB36398531',
 'MLB54448099',
 'MLB48677687',
 'MLB19552926',
 'MLB27839322',
 'MLB28002506',
 'MLB56994543',
 'MLB29568434',
 'MLB26446377',
 'MLB38750127',
 'MLB54236783',
 'MLB41626835',
 'MLB24455199',
 'MLB57356995',
 'MLB27985244',
 'MLB28682642',
 'MLB43616708',
 'MLB27769043',
 'MLB34345595',
 'MLB28754492',
 'MLB61666608',
 'MLB40002972',
 'MLB25735509',
 'MLB41799760',
 'MLB39171159',
 'MLB28037061',
 'MLB28834431',
 'MLB51925280',
 'MLB35985980',
 'MLB23222747',
 'MLB45867674',
 'MLB36956947',
 'MLB27238649',
 'MLB20049271',
 'MLB20545758',
 'MLB29417181',
 'MLB26085789',
 'MLB42639627',
 'MLB39983380',
 'MLB20662835',
 'MLB26051587',
 'MLB52139663',
 'MLB22145672',
 'MLB24198079',
 'MLB37791461',
 'MLB24228530',
 'MLB58376900',
 'MLB36411103',
 'MLB36034704',
 'MLB23942605',
 'MLB44845954',
 'MLB22031377',
 'MLB40543976',
 'MLB48710929',
 'MLB60176792',
 'MLB48848053',
 'MLB38347793',
 'MLB38264333',
 'MLB385

In [40]:
smartObjectsReviews = ( avaliacoes[avaliacoes["id"].isin(smartObjects)]
                       .drop_duplicates()
                       .reset_index(drop=True)
                       .dropna() )
smartObjectsReviews

,texto,nota,id,data
0,E muito prático. Gostei.,5,MLB36333923,13 dez. 2024
1,Ajuda muito em viagens.,5,MLB36333923,16 jan. 2025
2,Muito top.,5,MLB36333923,29 mar. 2025
3,Me atende produto muito bom bom para o que pre...,5,MLB36333923,08 mai. 2025
4,Gostei muito 👏👏👏👏.,5,MLB36333923,24 jul. 2025
...,...,...,...,...
4322,Atendeu super bem para controles de ventilador...,5,MLB50911897,28 nov. 2025
4324,Produto excelente e facil instalacao.,5,MLB50911897,01 dez. 2025
4325,"Simples, rápido e eficiente. Produto muito bom...",5,MLB50911897,25 out. 2025
4326,É elogiado por sua facilidade de configuração ...,5,MLB50911897,20 nov. 2025


In [41]:
smartObjectsQuestions = (perguntas[perguntas["id"].isin(smartObjects)]
                         .drop_duplicates()
                         .reset_index(drop=True))
smartObjectsQuestions

,pergunta,resposta,id,data
0,Qual tamanho do cabo 12V,"Olá, boa tarde! Cerca de 2M!",MLB36333923,28/08/2025
1,Ela não congela só esfria,"Olá, boa tarde! Isso, ela é uma geladeira, dif...",MLB36333923,15/07/2025
2,"Boa tarde Estou precisando de duas placas , pr...","Olá, bom dia! Infelizmente não temos peças de ...",MLB36333923,14/07/2025
3,Tem o link só do cabo de alimentação dessa mes...,"Olá boa noite, No momento não temos estoque so...",MLB36333923,12/07/2025
4,Boa tarde preciso da 220 mas somente a 110 est...,"Oi, Luiz acurcio Abreu! Tudo bem? Neste anúnci...",MLB54448099,13/11/2025
...,...,...,...,...
14640,"Olá vendedor, a lâmpada é compatível com rede ...","Olá, A lâmpada Xiaomi Inteligente Wifi Smart L...",MLB37232290,02/12/2025
14641,É 127v ou 220v?,"Boa tarde, agradecemos pelo contato! Ela é biv...",MLB43919988,10/11/2025
14642,Posso sintonizar a cor pelo app?,"Boa tarde, agradecemos pelo contato! Sim, pelo...",MLB43919988,10/11/2025
14643,"Olá, consigo deixar sempre no branco?","É só altera de cor pelo aplicativo, a última c...",MLB43919988,30/05/2025


In [55]:
def separa_data_ano(data):
    return data[-4:]

def separa_data_mes_q(data):
    return data[-7:-5]

def separa_data_mes_r(data):
    return data[-9:-6]

In [56]:
smartObjectsQuestions["ano_postagem"] = smartObjectsQuestions["data"].apply(separa_data_ano)
smartObjectsQuestions["mes_postagem"] = smartObjectsQuestions["data"].apply(separa_data_mes_q)
smartObjectsReviews["ano_postagem"] = smartObjectsReviews["data"].apply(separa_data_ano)
smartObjectsReviews["mes_postagem"] = smartObjectsReviews["data"].apply(separa_data_mes_r)
smartObjectsReviews

,texto,nota,id,data,ano_postagem,mes_postagem
0,E muito prático. Gostei.,5,MLB36333923,13 dez. 2024,2024,dez
1,Ajuda muito em viagens.,5,MLB36333923,16 jan. 2025,2025,jan
2,Muito top.,5,MLB36333923,29 mar. 2025,2025,mar
3,Me atende produto muito bom bom para o que pre...,5,MLB36333923,08 mai. 2025,2025,mai
4,Gostei muito 👏👏👏👏.,5,MLB36333923,24 jul. 2025,2025,jul
...,...,...,...,...,...,...
4322,Atendeu super bem para controles de ventilador...,5,MLB50911897,28 nov. 2025,2025,nov
4324,Produto excelente e facil instalacao.,5,MLB50911897,01 dez. 2025,2025,dez
4325,"Simples, rápido e eficiente. Produto muito bom...",5,MLB50911897,25 out. 2025,2025,out
4326,É elogiado por sua facilidade de configuração ...,5,MLB50911897,20 nov. 2025,2025,nov


In [ ]:
smartObjectsQuestions["mes_postagem"].value_counts()



mes_postagem
11    3626
10    3159
08    1758
05    1648
07    1465
09    1376
06    1068
12     374
04     171
Name: count, dtype: int64